In [3]:
import pandas as pd 
import numpy as np
import plotly.express as px
from datetime import datetime
import warnings
import seaborn as sns
import matplotlib.pyplot as plt


warnings.filterwarnings('ignore')

C:\Users\sarth\AppData\Local\Temp\ipykernel_15728\430151891.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
# Loading the data files.
dx_map = pd.read_csv("adsa_interview/data/dx_map.csv")
dx = pd.read_csv('adsa_interview/data/dx.csv')
demo = pd.read_csv('adsa_interview/data/demo.csv')

# 1) Exploratory Data Analysis

In [5]:
# Getting information about data.
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5003 entries, 0 to 5002
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   patient_id  5003 non-null   int64  
 1   site        5003 non-null   object 
 2   birth_date  5003 non-null   object 
 3   gender      5000 non-null   object 
 4   death_date  703 non-null    object 
 5   ethnicity   4710 non-null   object 
 6   zipcode     4805 non-null   float64
dtypes: float64(1), int64(1), object(5)
memory usage: 273.7+ KB


birth_date and death_date should have datetime data types.

In [6]:
# Converting the dates into appropriate format.
demo['birth_date'] = demo['birth_date'].astype('datetime64[ns]')
demo['death_date'] = demo['death_date'].astype('datetime64[ns]')

In [7]:
# Getting insights (5 number summary) about demographic data
demo.describe() 

,patient_id,birth_date,death_date,zipcode
count,5003.000000,5003,703,4805.000000
mean,5040.276034,1962-06-14 18:20:04.557265632,2004-06-24 09:31:29.615931776,55477.975234
min,5.000000,1895-07-28 00:00:00,1967-02-11 00:00:00,602.000000
25%,2573.000000,1944-08-17 12:00:00,2001-01-01 00:00:00,23608.000000
50%,5040.000000,1960-01-12 00:00:00,2001-01-01 00:00:00,60632.000000
75%,7515.500000,1979-11-12 12:00:00,2010-06-26 12:00:00,89030.000000
max,10000.000000,2023-11-15 00:00:00,2024-01-13 00:00:00,99654.000000
std,2872.423316,NaN,NaN,32585.493893


In [8]:
# Getting insights (5 number summary) about diagnostic data (dx)
dx.describe()

,patient_id
count,10018.000000
mean,5267.632961
std,2871.999386
min,6.000000
25%,2939.000000
50%,5318.000000
75%,7581.000000
max,9995.000000


In [9]:
# Getting insights about demographic data (dx_map)
dx_map.describe()

,condition,dx_code,dx_method
count,1413,1413,1413
unique,10,1255,3
top,Malignancy,E11.351,I10
freq,700,4,576


From the table we can see Malignancy is the most common condition that a patient has.

We found out that all of the conditions mentioned are starting with a capital letter and thus the query becomes case sensitive.

In [10]:
# Merging all the tables into one to extract the cohort of living patients with age equal to or more than 18.
df_merged = pd.merge(pd.merge(demo, dx, on='patient_id'), dx_map, on='dx_code')

# Removing unnecessary joined columns
df_merged.drop('dx_method_y', axis=1, inplace=True)

# Renaming the column
df_merged.rename({"dx_method_x":"dx_method"}, axis=1, inplace=True)

In [11]:
# Exploring the death_date to see if it is greater than the current time.

sum(df_merged['death_date'] > datetime.now())

# This means that the synthetic data does not have a death_date greater than the current date.

0

In [12]:
# Calculate age for each patient
df_merged['age'] = pd.Series([(datetime.now().year - bd.year) for bd in df_merged['birth_date']])

# Filter for living patients who are 18 years or older
prospective_study_df = df_merged[(df_merged['age'] >= 18) & (df_merged['death_date'].isnull())]

# Display the resulting DataFrame
prospective_study_df.head()

,patient_id,site,birth_date,gender,death_date,ethnicity,zipcode,dx_date,dx_code,dx_method,condition,age
1,1798,Site 2,1960-08-31,FEMALE,NaT,Not Hispanic or Latino,33024.0,2007-03-18,780.79,I9,Fatigue,64
2,8182,Site 4,1950-10-29,FEMALE,NaT,Not Hispanic or Latino,21045.0,2020-01-25,E11.9,I10,Diabetes,74
3,8182,Site 4,1950-10-29,FEMALE,NaT,Not Hispanic or Latino,21045.0,2020-01-25,E11.9,I10,Diabetes,74
4,8182,Site 4,1950-10-29,FEMALE,NaT,Not Hispanic or Latino,21045.0,2019-07-17,E66.9,I10,Obesity,74
5,8182,Site 4,1950-10-29,FEMALE,NaT,Not Hispanic or Latino,21045.0,2019-11-04,F32.9,I10,Depression,74


In [13]:
prospective_study_df.to_csv('cohort.csv', index=False)

In [14]:
prospective_study_df['age'].describe()
# As we can see we have some unrealistic values for the age which are 129. Let's find out the distribution of the age field.

count    7464.000000
mean       66.664523
std        19.817180
min        18.000000
25%        53.000000
50%        69.000000
75%        81.000000
max       129.000000
Name: age, dtype: float64

In [15]:
fig = px.box(prospective_study_df, x='age')

fig.update_layout(title='Age Distribution', title_x=0.5)
fig.show()

As can be seen from the above box plot that there is only one individual with an age of 129 and the upper fence of the boxplot ends around 105. We would remove this outlier from the dataset.


Assumption: The outlier is being removed in the assumption that the age data here has error in it and a general life expectancy of an individual is around 80-100 years.

In [16]:
prospective_study_df = prospective_study_df[prospective_study_df['age']<129]

In [17]:
# Getting the 5 number summary of the dataset.
prospective_study_df.describe()

,patient_id,birth_date,death_date,zipcode,age
count,7460.000000,7460,0,7383.000000,7460.000000
mean,5253.959651,1957-11-12 01:05:26.219839104,NaT,53742.090614,66.631099
min,6.000000,1919-05-01 00:00:00,NaT,603.000000,18.000000
25%,2842.500000,1943-05-03 00:00:00,NaT,19129.000000,53.000000
50%,5366.500000,1955-02-28 00:00:00,NaT,60629.000000,69.000000
75%,7524.000000,1971-03-28 00:00:00,NaT,85705.000000,81.000000
max,9995.000000,2006-12-26 00:00:00,NaT,99654.000000,105.000000
std,2898.533752,NaN,NaN,33457.835110,19.769834


In [18]:
# Check for missing data
prospective_study_df.isna().sum()

patient_id       0
site             0
birth_date       0
gender           0
death_date    7460
ethnicity        7
zipcode         77
dx_date          0
dx_code          0
dx_method        0
condition        0
age              0
dtype: int64

As we can see that we have some missing data majorly corresponding to ethnicity and zipcode.

Death_date is expected to have null values as we are expecting the cohort to not include any value in the death column and thus removing it.

In [19]:
prospective_study_df.drop('death_date', axis=1, inplace=True)

In [20]:
# Printing the length of the dataset
print(f'Length of Dataset before removing duplicates: {len(prospective_study_df)}')

# Checking for duplicate data in the table.
prospective_study_df.drop_duplicates(inplace=True)

print(f'Length of Dataset after removing duplicates: {len(prospective_study_df)}')

Length of Dataset before removing duplicates: 7460
Length of Dataset after removing duplicates: 6846


From the analysis, it is clear that there were a few duplicates in the dataset and those are being dropped.

During our analysis, we found out that the gender has case sensitivity and for a few records we had them written in lower case with first letter in upper case.

In [21]:
prospective_study_df['gender'].value_counts()

gender
FEMALE    3820
MALE      2989
Male        19
Female      18
Name: count, dtype: int64

In [22]:
prospective_study_df['gender'] = prospective_study_df['gender'].apply(lambda x: x.upper())
prospective_study_df['gender'].value_counts()

gender
FEMALE    3838
MALE      3008
Name: count, dtype: int64

In [23]:
# Visualization - Gender Distribution
fig_gender_distribution = px.pie(prospective_study_df, names='gender', title='Females have more health problems')
fig_gender_distribution.update_layout(title_x=0.5)
fig_gender_distribution.show()

It can be seen from the above distribution between genders that females are more prone to having these health problems than males.

In [24]:
# PLotting a distribution of categorical variables.
cols = ['site', 'dx_method', 'condition']

fig = px.bar(prospective_study_df, x=cols[0], color=cols[0])
fig.update_layout(title="Site distribution is imbalanced")
fig.show()

fig = px.bar(prospective_study_df, x=cols[1], color=cols[1])
fig.update_layout(title="Site distribution is imbalanced")
fig.show()


# 2) Relationship between Hypertension and Diabetes

In [25]:
prospective_study_df['patient_id'].nunique()

1277

In [26]:
# Filter data for patients with hypertension or diabetes diagnosis
hypertension_diabetes_df = prospective_study_df[prospective_study_df['condition'].isin(['Diabetes', 'Hypertension'])]

# Count of patients with hypertension, diabetes, or both
count_diagnoses = hypertension_diabetes_df.groupby('condition').size().reset_index(name='count')

# Visualization - Relationship between Hypertension and Diabetes
fig_diagnosis_relationship = px.bar(count_diagnoses, x='condition', y='count', color='condition',
                                    labels={'condition': 'Condition of Patient', 'count': 'Count'},
                                    title='Patients with Hypertension and Diabetes overlap')

print(f'Total number of records: {len(hypertension_diabetes_df)}')
fig_diagnosis_relationship.update_layout(title_x = 0.5)
fig_diagnosis_relationship.show()

Total number of records: 1446


As we can see that there is an overlap between the patients with diabetes and hypertension and thus we can consider that there are patients who have diabetes as well as hypertension.

PS: In my study, I found out that the dx_code I9 represents Hypertension and the code E11.9 represent diabetes.

In [27]:
# Assuming hypertension_diabetes_df is the DataFrame containing relevant data
result_df = (hypertension_diabetes_df.groupby('patient_id')
             .apply(lambda group: 'Hypertension' in group['condition'].values and 'Diabetes' in group['condition'].values)
             .reset_index(name='both_diagnoses'))

cnt = result_df['both_diagnoses'].sum()
print(f"Count of patients with both diagnoses: {cnt}")

Count of patients with both diagnoses: 158


Hence, there are 158 patients with both diabetes and hypertension.

Having this information in hand now, it is very imperative to check the age distribution of these patients. Let us create two dataframes corresponding to each of the condition of patients.

In [28]:
Diabetes_df = hypertension_diabetes_df[hypertension_diabetes_df['condition']=='Diabetes']
Hypertension_df = hypertension_diabetes_df[hypertension_diabetes_df['condition']=='Hypertension']

In [ ]:
# Plot a distribution of age from both of the dataframes.
f1 = px.histogram(Diabetes_df, x='age', labels={"age": 'Age of Patients with Diabetes'})
f1.show()

f2 = px.histogram(Hypertension_df, x='age', labels={"age": 'Age of Patients with Hypertension'})
f2.show()

As we can see from the above distribution that there are more people suffering from Hypertension than Diabetes above the age of 75. Also, we can see the ages are not distributed normally.

As a matter of fact, it is crucial to also check what all other conditions do these patients suffer from which is an imperative factor in predicting the future conditions of those patients.

In [ ]:
patient_ids = hypertension_diabetes_df['patient_id'].unique()

prospective_study_df2 = prospective_study_df[prospective_study_df['patient_id'].isin(patient_ids)]
prospective_study_df2[~prospective_study_df2['condition'].isin(['Hypertension', 'Diabetes'])]['condition'].unique()

array(['Obesity', 'Depression', 'Anxiety', 'Pain, Back',
       'Arrhythmia, Cardiac', 'Dyspnea', 'Fatigue', 'Malignancy'],
      dtype=object)

Hence patients with conditions like Obesity or Depression should be adviced to watch out for Diabetes and Hypertension as a potential health problem and must take precautions.

Other work - 

Number of people with hypertension and diabetes in contrast with people with other diseases. 